In [ ]:
print([ i.replace('\'','').strip() for i in train['tokens'][0][1:-1].split('\',')])

In [19]:
import torch
import pickle
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel


config_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(config_name)
model = AutoModel.from_pretrained(config_name)


def create_claim_encodings(file_path,output_path):
    with open(file_path,'r') as f:
        claim_descriptions = f.readlines()
    claim_descriptions[0] = claim_descriptions[0][1:]
    descriptions = clean_desc(claim_descriptions)
    tokens = tokenizer(descriptions, padding='max_length', max_length = 25, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)

    with open(output_path,'wb') as f:
        pickle.dump(outputs.last_hidden_state,f)
    
    return outputs.last_hidden_state

def clean_desc(claim_descriptions):
    desc_fin = []
    for i in range(len(claim_descriptions)):
        desc = claim_descriptions[i].strip().strip(',').strip('\"').lower().split()
        fin = []
        for wrd in desc:
            if wrd not in stopwords.words('english'):
                fin.append(wrd)
                
        fin = ' '.join(fin)
        desc_fin.append(fin)
    return desc_fin


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
descs = create_claim_encodings(file_path='new_descriptions.txt',output_path = 'claim_desc_new_vecs_base.pkl')

In [21]:
descs.shape

torch.Size([18, 25, 768])